<a href="https://colab.research.google.com/github/vishnushukla1729/GenAI---BERT-and-T5-Summariation/blob/main/Source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Feb  4 22:31:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 33.4 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
# Import necessary libraries
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd
from evaluate import load
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

# Download necessary NLTK resources
nltk.download("punkt")

# Set device for PyTorch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: cuda


In [61]:
# Cell 1: Define Common Transformer Building Blocks
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # shape (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        x = x + self.pe[:, :x.size(1)]
        return x

# Scaled Dot-Product Attention
def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        attn = dropout(attn)
    output = torch.matmul(attn, value)
    return output

# Multi-head Attention Module
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % heads == 0, 'd_model must be divisible by heads'
        self.d_k = d_model // heads
        self.heads = heads
        self.linear_layers = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(3)])
        self.output_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        nbatches = query.size(0)
        # 1) Do all linear projections and split into heads
        query, key, value = [l(x).view(nbatches, -1, self.heads, self.d_k).transpose(1,2)
                             for l, x in zip(self.linear_layers, (query, key, value))]
        # 2) Apply attention on all the projected vectors in batch
        x = attention(query, key, value, mask=mask, dropout=self.dropout)
        # 3) Concatenate heads and put through final linear layer
        x = x.transpose(1,2).contiguous().view(nbatches, -1, self.heads * self.d_k)
        return self.output_linear(x)

# Feed Forward Network
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.linear2(self.dropout(F.relu(self.linear1(x))))

# Transformer Encoder Block
class TransformerEncoderBlock(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout=0.1):
        super(TransformerEncoderBlock, self).__init__()
        self.attention = MultiHeadAttention(heads, d_model, dropout)
        self.ffn = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # Self-attention with residual and layer norm
        attn_out = self.attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_out))
        # Feed forward network with residual and layer norm
        ffn_out = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_out))
        return x

print('Common Transformer building blocks defined')

Common Transformer building blocks defined


In [62]:
# Cell 2: Define a simplified BERT model from scratch

class SimpleBERT(nn.Module):
    def __init__(self, vocab_size, d_model=128, max_len=512, num_layers=2, heads=4, d_ff=256, num_classes=2, dropout=0.1):
        super(SimpleBERT, self).__init__()
        self.token_embeddings = nn.Embedding(vocab_size, d_model)
        self.position_embeddings = PositionalEncoding(d_model, max_len)
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderBlock(d_model, heads, d_ff, dropout) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, mask=None):
        # input_ids: (batch_size, seq_len)
        x = self.token_embeddings(input_ids)  # (batch_size, seq_len, d_model)
        x = self.position_embeddings(x)
        for layer in self.encoder_layers:
            x = layer(x, mask)
        # Use the representation of the first token ([CLS]) for classification
        cls_rep = x[:, 0, :]
        cls_rep = self.norm(cls_rep)
        cls_rep = self.dropout(cls_rep)
        logits = self.classifier(cls_rep)
        return logits

# Instantiate and print a summary for SimpleBERT
vocab_size = 30522  # typical BERT vocab size
model_bert = SimpleBERT(vocab_size=vocab_size, d_model=128, max_len=128, num_layers=2, heads=4, d_ff=256, num_classes=2)
print('SimpleBERT model defined')

# Dummy test for SimpleBERT
input_ids = torch.randint(0, vocab_size, (2, 20))  # batch of 2 sequences of length 20
logits = model_bert(input_ids)
print('SimpleBERT logits shape: ' + str(logits.shape))

# Cell 3: Define a simplified T5 model from scratch (Encoder-Decoder architecture)

class TransformerDecoderBlock(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout=0.1):
        super(TransformerDecoderBlock, self).__init__()
        self.self_attention = MultiHeadAttention(heads, d_model, dropout)
        self.cross_attention = MultiHeadAttention(heads, d_model, dropout)
        self.ffn = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        # self-attention with causal mask for decoder
        self_attn = self.self_attention(x, x, x, mask=tgt_mask)
        x = self.norm1(x + self.dropout(self_attn))
        # cross attention over encoder output
        cross_attn = self.cross_attention(x, encoder_output, encoder_output, mask=src_mask)
        x = self.norm2(x + self.dropout(cross_attn))
        # feed forward network
        ffn_out = self.ffn(x)
        x = self.norm3(x + self.dropout(ffn_out))
        return x

class SimpleT5(nn.Module):
    def __init__(self, vocab_size, d_model=128, max_len=512, num_encoder_layers=2, num_decoder_layers=2, heads=4, d_ff=256, dropout=0.1):
        super(SimpleT5, self).__init__()
        self.encoder_token_embeddings = nn.Embedding(vocab_size, d_model)
        self.encoder_position_embeddings = PositionalEncoding(d_model, max_len)
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderBlock(d_model, heads, d_ff, dropout) for _ in range(num_encoder_layers)
        ])

        self.decoder_token_embeddings = nn.Embedding(vocab_size, d_model)
        self.decoder_position_embeddings = PositionalEncoding(d_model, max_len)
        self.decoder_layers = nn.ModuleList([
            TransformerDecoderBlock(d_model, heads, d_ff, dropout) for _ in range(num_decoder_layers)
        ])

        self.output_linear = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def encode(self, input_ids, mask=None):
        x = self.encoder_token_embeddings(input_ids)
        x = self.encoder_position_embeddings(x)
        for layer in self.encoder_layers:
            x = layer(x, mask)
        return x

    def decode(self, target_ids, encoder_output, src_mask=None, tgt_mask=None):
        x = self.decoder_token_embeddings(target_ids)
        x = self.decoder_position_embeddings(x)
        for layer in self.decoder_layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return x

    def forward(self, input_ids, target_ids, src_mask=None, tgt_mask=None):
        encoder_output = self.encode(input_ids, src_mask)
        decoder_output = self.decode(target_ids, encoder_output, src_mask, tgt_mask)
        # generate logits for vocabulary
        logits = self.output_linear(decoder_output)
        return logits

# Instantiate and print a summary for SimpleT5
model_t5 = SimpleT5(vocab_size=vocab_size, d_model=128, max_len=128, num_encoder_layers=2, num_decoder_layers=2, heads=4, d_ff=256)
print('SimpleT5 model defined')

# Dummy test for SimpleT5
input_ids = torch.randint(0, vocab_size, (2, 20))  # encoder input
# target_ids: start token assumed as 0
target_ids = torch.randint(0, vocab_size, (2, 15))
logits_t5 = model_t5(input_ids, target_ids)
print('SimpleT5 logits shape: ' + str(logits_t5.shape))
print('Models for BERT and T5 have been defined and tested on dummy data.')

SimpleBERT model defined
SimpleBERT logits shape: torch.Size([2, 2])
SimpleT5 model defined
SimpleT5 logits shape: torch.Size([2, 15, 30522])
Models for BERT and T5 have been defined and tested on dummy data.


In [66]:
def build_vocab(text):
    tokens = text.lower().split()
    vocab = {'<s>': 0, '</s>': 1}  # start and end tokens
    idx = 2
    for token in tokens:
        if token not in vocab:
            vocab[token] = idx
            idx += 1
    return vocab

def tokenize(text, vocab):
    tokens = text.lower().split()
    return [vocab.get(token, vocab['</s>']) for token in tokens]

def detokenize(token_ids, inv_vocab):
    tokens = [inv_vocab.get(tid, '') for tid in token_ids if tid not in (0, 1)]
    return ' '.join(tokens)

# Define the sample paragraph
paragraph = ("Artificial Intelligence has revolutionized many industries. "
             "Deep learning techniques are now widely adopted in tasks such as speech recognition, "
             "computer vision, and natural language processing. The rapid development of AI technologies "
             "has led to significant breakthroughs and continues to drive innovative solutions.")

# Build vocabulary and inverse vocabulary from the paragraph
vocab = build_vocab(paragraph)
inv_vocab = {v: k for k, v in vocab.items()}

# Convert paragraph to input_ids and pad to 128 tokens for simplicity
input_ids = tokenize(paragraph, vocab)
if len(input_ids) < 128:
    input_ids = input_ids + [vocab['</s>']] * (128 - len(input_ids))
else:
    input_ids = input_ids[:128]
input_ids_tensor = torch.tensor([input_ids])  # batch size of 1

# Define a simple greedy decoding function for the SimpleT5 model
def greedy_decode(model, input_ids, max_len=20, start_token=0, end_token=1):
    model.eval()
    device = next(model.parameters()).device
    input_ids = input_ids.to(device)

    # Initialize target sequence with the start token
    target_ids = torch.tensor([[start_token]], device=device)

    for _ in range(max_len):
        logits = model(input_ids, target_ids)
        next_token_logits = logits[:, -1, :]
        next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)
        target_ids = torch.cat([target_ids, next_token_id], dim=1)

        if (next_token_id == end_token).all():
            break

    return target_ids

# Instantiate the SimpleT5 model (untrained)
vocab_size = len(vocab)  # Adjusted vocab size based on the paragraph
model_t5 = SimpleT5(vocab_size=vocab_size, d_model=128, max_len=128, num_encoder_layers=2, num_decoder_layers=2, heads=4, d_ff=256)

# Move model to the correct device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_t5.to(device)

# Generate the summary using the greedy decode function
generated_ids = greedy_decode(model_t5, input_ids_tensor, max_len=20, start_token=vocab['<s>'], end_token=vocab['</s>'])
generated_ids_list = generated_ids[0].tolist()
summary = detokenize(generated_ids_list, inv_vocab)

print('Input Paragraph:')
print(paragraph)
print('\nGenerated Summary:')
print(summary)

Input Paragraph:
Artificial Intelligence has revolutionized many industries. Deep learning techniques are now widely adopted in tasks such as speech recognition, computer vision, and natural language processing. The rapid development of AI technologies has led to significant breakthroughs and continues to drive innovative solutions.

Generated Summary:
vision, intelligence significant ai vision, intelligence ai vision, the continues continues continues continues continues continues continues continues continues continues continues


In [67]:
from transformers import pipeline

# The paragraph to test both models (feel free to change as needed)
paragraph = (
    "Artificial Intelligence (AI) has revolutionized multiple industries. "
    "From healthcare to self-driving cars, innovations transform our world. "
    "Deep learning and natural language processing (NLP) have become foundational. "
    "Companies leverage these technologies to improve operational efficiency and create value. "
    "However, with rapid development come challenges in ethics and regulatory oversight."
)

# Print the paragraph for verification
print("Test Paragraph:")
print(paragraph)

Test Paragraph:
Artificial Intelligence (AI) has revolutionized multiple industries. From healthcare to self-driving cars, innovations transform our world. Deep learning and natural language processing (NLP) have become foundational. Companies leverage these technologies to improve operational efficiency and create value. However, with rapid development come challenges in ethics and regulatory oversight.


In [68]:
summarizer = pipeline("summarization", model="t5-small")

# Generate summary for the paragraph
summary = summarizer(paragraph, max_length=50, min_length=25, do_sample=False)
print("Summary:")
print(summary[0]['summary_text'])

Device set to use cuda:0


Summary:
Artificial Intelligence (AI) has revolutionized multiple industries . from healthcare to self-driving cars, innovations transform our world .


In [69]:

# For demonstration purposes, we're using a sentiment-analysis pipeline.
# In a real text classification task, you may want to use a model fine-tuned on a specific classification objective.
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Classify the input paragraph
classification = classifier(paragraph)
print("Classification:")
print(classification[0])

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


Classification:
{'label': 'POSITIVE', 'score': 0.9962367415428162}


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [5]:
model_ckpt = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [10]:
dataset_samsum = load_dataset("samsum")

corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [11]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [12]:
dataset_samsum["train"]["dialogue"][1]


'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great'

In [13]:
dataset_samsum["train"][1]["summary"]

'Olivia and Olivier are voting for liberals in this election. '

In [14]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [15]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [16]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [17]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [18]:
dataset_samsum_pt["train"]["input_ids"][1]

[0,
 673,
 31332,
 493,
 35,
 3394,
 32,
 47,
 3434,
 13,
 11,
 42,
 729,
 116,
 1437,
 50121,
 50118,
 33453,
 8538,
 35,
 12993,
 25,
 460,
 4,
 50121,
 50118,
 673,
 31332,
 493,
 35,
 1464,
 350,
 12846,
 50121,
 50118,
 33453,
 8538,
 35,
 2860,
 2]

In [19]:
dataset_samsum_pt["train"]["attention_mask"][1]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [20]:
dataset_samsum_pt["train"]["labels"][1]

[0, 673, 31332, 493, 8, 20544, 32, 3434, 13, 19561, 11, 42, 729, 4, 1437, 2]

In [21]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [31]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='bart-large-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=100,
    evaluation_strategy='steps', eval_steps=1000, save_steps=1e6,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:

trainer = Trainer(model=model, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

<ipython-input-32-9613924cd16f>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=trainer_args,


In [33]:
!pip install wandb
import wandb

# Initialize W&B
wandb.init(project="Source.ipynb")

In [34]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)


rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f'bart-large'])


 11%|█         | 45/410 [02:25<19:01,  3.13s/it]

In [ ]:
## Save model
model.save_pretrained("bart-large-samsum-model")

In [ ]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.json',
 'tokenizer/merges.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":4, "max_length": 128}



sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="bart-large-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda can't find Betty's number. Larry called Betty the last time they were at the park together. Hannah wants Amanda to text him instead of calling him.    She doesn't know Larry well, but he's nice and he's very nice.


In [ ]:
# Step 1: Setup environment and import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer
from transformers import get_linear_schedule_with_warmup
from datasets import load_dataset
import numpy as np

print('Step 1: Environment set up and libraries imported.')

# Step 2: Load a dataset for summarization - we use the XSum dataset from Hugging Face for demonstration
# The XSum dataset has articles and summaries
xsum_dataset = load_dataset('xsum', split='train[:1%]')  # using 1% for quick demonstration
print('Step 2: Loaded XSum dataset sample.')
print(xsum_dataset[0])

# Step 3: Preprocessing - tokenize input and summary
# We use BertTokenizer for both encoder and decoder sections for simplicity, though normally you'd use a different one for the decoder

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define maximum lengths
max_input_length = 256
max_summary_length = 64

def preprocess(sample):
    # Tokenize the source article
    inputs = tokenizer(sample['document'], truncation=True, padding='max_length', max_length=max_input_length, return_tensors='pt')
    # Tokenize the summary
    targets = tokenizer(sample['summary'], truncation=True, padding='max_length', max_length=max_summary_length, return_tensors='pt')

    # Squeeze to remove batch dimension, we'll add later
    inputs = {key: val.squeeze(0) for key, val in inputs.items()}
    targets = {key: val.squeeze(0) for key, val in targets.items()}

    return {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask'],
            'labels': targets['input_ids']}

# Preprocess a small batch
processed_dataset = [preprocess(x) for x in xsum_dataset]
print('Step 3: Preprocessing done on small sample.')

# Step 4: Model Architecture
# We build a summarization model using a Bert encoder and a Transformer decoder

class Bert2TransformerSummarizer(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', d_model=768, nhead=8, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
        super(Bert2TransformerSummarizer, self).__init__()
        # Load BERT as encoder
        self.encoder = BertModel.from_pretrained(bert_model_name)
        # Freeze BERT parameters if desired
        # for param in self.encoder.parameters():
        #     param.requires_grad = False

        # Define the decoder: use PyTorch Transformer decoder
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_decoder_layers)

        # Embedding for decoder - using the same vocab size as BERT
        self.decoder_embedding = nn.Embedding(tokenizer.vocab_size, d_model)

        # Final linear layer to project decoder outputs to vocabulary logits
        self.fc_out = nn.Linear(d_model, tokenizer.vocab_size)

        # Positional encoding (simplified using learned embeddings)
        self.positional_embedding = nn.Embedding(512, d_model)  # assume max sequence length 512

    def forward(self, src_input_ids, src_attention_mask, tgt_input_ids):
        # Encode using BERT encoder
        encoder_outputs = self.encoder(input_ids=src_input_ids, attention_mask=src_attention_mask)
        memory = encoder_outputs.last_hidden_state.transpose(0,1)  # shape [seq_len, batch, d_model]

        # Embed decoder inputs
        tgt_embeddings = self.decoder_embedding(tgt_input_ids)  # [batch, tgt_seq, d_model]
        batch_size, tgt_seq, d_model = tgt_embeddings.size()
        # Add positional embeddings
        positions = torch.arange(0, tgt_seq).unsqueeze(0).expand(batch_size, tgt_seq).to(tgt_embeddings.device)
        tgt_embeddings = tgt_embeddings + self.positional_embedding(positions)
        tgt_embeddings = tgt_embeddings.transpose(0,1)  # shape [tgt_seq, batch, d_model]

        # Create target mask for transformer decoder (subsequent mask so predictions don't look ahead)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_seq).to(tgt_embeddings.device)

        # Decode
        outputs = self.decoder(tgt=tgt_embeddings, memory=memory, tgt_mask=tgt_mask)
        outputs = outputs.transpose(0,1)  # [batch, tgt_seq, d_model]
        logits = self.fc_out(outputs)  # [batch, tgt_seq, vocab_size]
        return logits

# Instantiate the model
model = Bert2TransformerSummarizer()
print('Step 4: Model built using Bert encoder and Transformer decoder.')

# Step 5: Training Loop (simplified demonstration)

# Use one sample batch from the processed data for demonstration
batch = processed_dataset[0]
# Add batch dimension
src_input_ids = batch['input_ids'].unsqueeze(0)  # shape [1, seq_len]
src_attention_mask = batch['attention_mask'].unsqueeze(0)
# For target, shifting right, here we use same tokens for input and labels for demo
# Create a simple mechanism: input to decoder is summary tokens shifted right
labels = batch['labels']
# For decoder input, prepend [CLS] token id (for demonstration) and remove last token
cls_token_id = tokenizer.cls_token_id if tokenizer.cls_token_id is not None else tokenizer.convert_tokens_to_ids('[CLS]')

tgt_input_ids = torch.cat([torch.tensor([cls_token_id]), labels[:-1]])
# Add batch dimension
tgt_input_ids = tgt_input_ids.unsqueeze(0)

# Forward pass
logits = model(src_input_ids, src_attention_mask, tgt_input_ids)

print('Step 5: Completed a forward pass through the model.')
print('Logits shape:', logits.shape)

# Step 6: Define a basic training loop framework

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-5)

# Dummy training loop with one iteration for demonstration
model.train()
optimizer.zero_grad()

# Shift logits and labels for loss computation: logits for each token and corresponding label
# Flatten the logits, expected shape: [batch*tgt_seq, vocab_size]
logits_flat = logits.reshape(-1, logits.size(-1))
# The target labels are the original summary tokens (not including the prepended token)
# but here we are using the entire labels for simplicity
loss = criterion(logits_flat, labels.unsqueeze(0).reshape(-1))
loss.backward()
optimizer.step()

print('Step 6: Completed one training step with loss:', loss.item())

print('BERT Summarizer Demonstration Completed.')

print('done')

Step 1: Environment set up and libraries imported.


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Step 3: Preprocessing done on small sample.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Step 4: Model built using Bert encoder and Transformer decoder.
Step 5: Completed a forward pass through the model.
Logits shape: torch.Size([1, 64, 30522])
Step 6: Completed one training step with loss: 10.490785598754883
BERT Summarizer Demonstration Completed.
done
